# Classification de texte avec TF-IDF et SVM linéaire

Ce notebook détaille un pipeline complet de classification de texte utilisant le prétraitement linguistique avec SpaCy, la vectorisation TF-IDF et un classifieur SVM linéaire (LinearSVC). Chaque étape est expliquée, du nettoyage à la génération du fichier de soumission.

---

## Concepts clés

- **TF-IDF (Term Frequency-Inverse Document Frequency)** : transforme chaque texte en un vecteur de nombres, reflétant l'importance des mots dans le corpus.
- **SVM linéaire (Support Vector Machine)** : algorithme de classification supervisée qui cherche l'hyperplan séparant au mieux les classes dans l'espace vectoriel.
- **Prétraitement linguistique** : nettoyage, lemmatisation, suppression des stopwords pour améliorer la qualité des features.

---

In [ ]:
# Importation des bibliothèques nécessaires
import json
import pandas as pd
import numpy as np
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## 1. Chargement et nettoyage des données

On charge les données d'entraînement depuis un fichier JSON. On remplace les valeurs manquantes dans la colonne 'description' par une chaîne vide pour éviter les erreurs lors du traitement.

In [ ]:
with open('train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
df = pd.DataFrame(train_data)
df['description'] = df['description'].fillna("")

## 2. Prétraitement linguistique avec SpaCy

On utilise SpaCy pour nettoyer et lemmatiser les textes :
- Suppression du HTML et des URLs
- Mise en minuscules
- Lemmatisation (réduction à la racine)
- Suppression des stopwords et tokens non alphabétiques

Ce prétraitement réduit la dimensionnalité et améliore la qualité des features pour le modèle.

In [ ]:
nlp = spacy.load('en_core_web_sm')

def clean_and_tokenize(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = text.lower().strip()
    doc = nlp(text)
    tokens = [tok.lemma_ for tok in doc if tok.is_alpha and not tok.is_stop]
    return " ".join(tokens)

df['Clean'] = df['description'].apply(clean_and_tokenize)

## 3. Chargement des labels

On associe à chaque texte sa catégorie cible pour l'apprentissage supervisé.

In [ ]:
labels_df = pd.read_csv('train_label.csv')
df = df.merge(labels_df, on='Id')

## 4. Vectorisation des textes avec TF-IDF

On transforme les textes en vecteurs de nombres avec `TfidfVectorizer` :
- Chaque texte devient un vecteur de taille `max_features` (ici 20 000)
- Chaque dimension correspond à un mot ou une paire de mots (n-grammes)
- La valeur est le score TF-IDF du mot dans le texte

Mathématiquement, le TF-IDF d'un mot $w$ dans un document $d$ est :
$$ \text{tfidf}(w, d) = tf(w, d) \times \log\left(\frac{N}{df(w)}\right) $$
où $tf(w, d)$ est la fréquence du mot dans le document, $N$ le nombre total de documents, $df(w)$ le nombre de documents contenant $w$.

In [ ]:
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2), min_df=5, norm='l2')
X_tfidf = tfidf.fit_transform(df['Clean'])
y = df['Category']

## 5. Séparation train/validation

On sépare les données en un ensemble d'entraînement (80%) et de validation (20%) pour évaluer la capacité de généralisation du modèle. On stratifie pour conserver la proportion des classes.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_tfidf, y, test_size=0.2, stratify=y, random_state=42
)

## 6. Entraînement du modèle SVM linéaire

Le SVM linéaire cherche l'hyperplan qui maximise la marge entre les classes dans l'espace vectoriel TF-IDF.

Mathématiquement, le SVM résout :
$$ \min_{w, b} \frac{1}{2} \|w\|^2 + C \sum_i \xi_i $$
sous contrainte $y_i (w^T x_i + b) \geq 1 - \xi_i$, où $\xi_i$ sont les variables d'écart (slack variables), $C$ contrôle la régularisation.

In [ ]:
svm_clf = LinearSVC(C=1.0, max_iter=5000)
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_val)

## 7. Évaluation du modèle

On prédit les classes sur l'ensemble de validation et on affiche un rapport de classification (précision, rappel, F1-score).

In [ ]:
print(classification_report(y_val, y_pred))

## 8. Prédiction sur les données de test

On applique le même prétraitement et la même vectorisation aux textes de test, puis on prédit la catégorie pour chaque texte.

In [ ]:
with open('test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
test_df = pd.DataFrame(test_data)
test_df['Clean'] = test_df['description'].fillna("").apply(clean_and_tokenize)
X_test = tfidf.transform(test_df['Clean'])
preds = svm_clf.predict(X_test)

## 9. Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
template = pd.read_csv('template_submissions.csv')
template['Category'] = template['Id'].map(dict(zip(test_df['Id'], preds)))
template.to_csv('submission_svm.csv', index=False)
print("Fichier de soumission enregistré : 'submission_svm.csv'")

---

## Concepts mathématiques et conclusion

- **TF-IDF** : pondère l'importance des mots selon leur fréquence dans le document et leur rareté dans le corpus.
- **SVM linéaire** : cherche l'hyperplan qui sépare au mieux les classes dans l'espace vectoriel, en maximisant la marge.
- **Classification** : la sortie du modèle est la classe avec la plus grande distance à l'hyperplan de séparation.

Ce pipeline montre comment passer de textes bruts à des prédictions de classes avec un SVM, en utilisant des représentations vectorielles robustes.